2023-10-30

Continuing from here: [creating_3d_dataset](/Users/jonathan/mres_thesis/wine_analysis_hplc_uv/src/wine_analysis_hplc_uv/notebooks/creating_3d_dataset.ipynb)

This document will contain my PCA analysis of a sample HPLC-DAD dataset for insights, including rank estimation.

In [ ]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from eda_by_category_methods import DTWProcessing 
from scipy.signal import savgol_filter
from scipy import signal
import seaborn.objects as so
import seaborn as sns
from pybaselines import Baseline
from sklearn.decomposition import PCA

scaler = StandardScaler()

process = DTWProcessing()

In [ ]:
from wine_analysis_hplc_uv.notebooks.xgboost_modeling.dataextract import RawTestSet3DCreator
from wine_analysis_hplc_uv import definitions
rtsc = RawTestSet3DCreator(definitions.DB_PATH)

dset = rtsc.df
dset.head()

In [ ]:
d111=dset.loc[lambda df: df.code_wine.str.contains('111')].drop(['detection','code_wine','color','varietal','id','mins'],axis=1).reset_index(drop=True).rename_axis('i')

d111

First the data needs to be baseline corrected and smoothed.

In [ ]:
# melt
m_d111 = d111.melt(var_name='wavelength',value_name='abs', ignore_index=False).reset_index()
m_d111

In [ ]:
# display head of melted frame
m_d111.head()

In [ ]:
# plot raw
fig, ax= plt.subplots(figsize=(5,3), dpi=150)

p = m_d111.pipe(lambda x: sns.lineplot(data=x, hue='wavelength',x='i', y='abs', legend=False, ax=ax
                                       ))

In [ ]:
# smooth

def smooth(df, grouper: str, col: str, smoothed_colname: str, savgol_kws:dict=dict(window_length=5, polyorder=2)):
    """
    Smooth the signal with savgol filter.
    
    args:
    
    df: long format dataframe with a group label column
    grouper: column containing the labels of the groups to iterate over
    smoothed_colname: name of the output column
    savgol_kws: refer to scipy.signal.savgol_filter
    """
    
    df=(df
    .assign(
        **{
        smoothed_colname:lambda df: df
        .groupby(grouper)[col]
        .transform(
        lambda x: pd.Series(
            savgol_filter(
                x,
                **savgol_kws
                ),
                index=x.index
        )
        )
        }
        )
        )
    display(df)

    return df

m_d111 = smooth(m_d111, 'wavelength','abs', 'smooth', savgol_kws=dict(window_length=4, polyorder=2))
m_d111.groupby('wavelength').get_group('nm_256').pipe(lambda x: so.Plot(data=x, x='i',y='smooth').add(so.Line())).show()

In [ ]:
# bcorr

def baseline_subtract(df, col, grouper, baseline_corrected_name: str):
    """
    Baseline subtract
    
    args:
    
    df - long df with wavelengths as an id col.
    col - target column to be transformed
    grouper - column containing the group labels, i.e. wavelengths
    baseline_corrected_name - the name of the newly created column
    """

    df = (df
            .assign(**dict(bline=lambda df: 
        df.groupby(grouper)
        .smooth
        .transform(lambda x: pd.Series(Baseline(x.index).asls(
            x,
            max_iter=50,
            tol=1E-3,
            lam=1E6,
            )[0], index=x.index))
    ))
            .assign(**{baseline_corrected_name:lambda df: df.eval("smooth-bline")
            })
    )
    return df

m_d111 = baseline_subtract(m_d111, 'smooth', 'wavelength', 'bcorr')

display(m_d111.head())

# plot overlay of baseline and original signal
(m_d111
 .groupby('wavelength')
.get_group('nm_256')
.loc[:,['i','smooth','bline']]
.pipe(lambda df: df if display(df) else df) # display df
.melt(var_name='signal', value_name='abs', id_vars='i')
.pipe(lambda df: df if display(df) else df) # display df
.pipe(lambda df: so.Plot(data=df, x='i',y='abs',color='signal').add(so.Line()))
)

First the data needs to be scaled and centered. Also, subset to the region of interest, < 4000.

In [ ]:
# subset to <4000

m_d111_subset = m_d111.loc[m_d111.i<4000]

display(m_d111_subset.head())

(
    m_d111_subset
    .groupby('wavelength')
    .get_group('nm_256')
    .reset_index()
    .pipe(lambda x: so.Plot(data=x, x='i',y='bcorr').add(so.Line()
    )
    )
)

In [ ]:
def scale_and_center(df, col: str):
    # scale and center

    df=df.assign(scale_center=lambda df: scaler.fit_transform(df[col].values.reshape(-1,1)))
    display(df.head())
    df.info()
    
    return df

m_d111_subset = scale_and_center(m_d111_subset, 'bcorr')

m_d111_subset.groupby('wavelength').get_group('nm_256').pipe(lambda x: so.Plot(data=x, x='i',y='scale_center').add(so.Line()).show())


# PCA

@juan_mcriter_2020 says that PCA can be used to estimate the number of compounds in $X$. @nardecchia_2020 says that this is based on "the scree test for the number of factors". Plotting eigenvalues against components, the chemical rank is defined as the point at which the curve elbows.

Component selection is necessarily arbitrary, ergo I will define the threshold of variance % as greater than 1E-3

In [ ]:
def calculate_components(df):
    """
    Calculate the number of principal components in the dataset defined as the set of those containing greater than 0.0001 variance ratio.
    
    args
    
    df: an augmented dataframe with more rows than columns, i.e. observations x wavelengths
    """
    
    # initialise the PCA object and then fit transform on df
    pca = PCA()
    pca.fit_transform(df)
    
    # construct a DataFrame of two columns: n = the number of components; and var_ratio = the variance ratio explained by that component
    
    screeplot_data=pd.DataFrame(dict(n=np.arange(pca.n_components_)+1, var_ratio=pca.explained_variance_ratio_))

    # filter out components with less than 1E-3 variance ratio
    selected_components = screeplot_data.loc[lambda x: x.var_ratio>1E-3]

    # display the selected components table
    display(selected_components)

    # create the scree plot, marking the last retained component
    screeplot_data.pipe(lambda x: so.Plot(data=x.loc[0:20], x='n', y='var_ratio').add(so.Line()).add(so.Dot(marker='x'),data=selected_components.iloc[[-1]], x='n',y='var_ratio')).show()
    display()

    # number of components is equal to the number of rows of the selected_components table
    
    n_components = selected_components.shape[0]

    # display the nmber of components
    display(f"n components = {n_components}")

    return n_components

n_components = calculate_components(m_d111_subset.pivot_table(columns='wavelength', values='scale_center', index='i'))

Therefore for this dataset, the chemical rank is 6. This is very surprising, as I was expecting at least as many components as peaks.

## Counting Peaks

In [ ]:
# find the peaks defined as those as prominant as 2% of the maxima of the signal


m_d111_subset = m_d111_subset.assign(
    peaks=lambda df: df.groupby("wavelength")["bcorr"].transform(
        lambda x: x.iloc[signal.find_peaks(x, prominence=x.max()*0.02)[0]]
    )
)

display(m_d111_subset.head())

m_d111_subset.groupby("wavelength").get_group("nm_256").pipe(
    lambda df: so.Plot(df, x="i").add(so.Line(), y="bcorr").add(so.Dot(), y="peaks")
)

In [ ]:
# a prominence value of 2 is appropriate for 256nm, but is it appropriate for all wavelengths?

(m_d111_subset
.loc[lambda df: df
     .wavelength.isin(['nm_190','nm_256','nm_400'])]
.pipe(
    lambda df: so.Plot(df, x="i")
    .facet('wavelength')
    .share(y=False)
    .add(so.Line(), y="bcorr")
    .add(so.Dot(), y="peaks")
    .layout(size=(15,3))
))

In [ ]:
m_d111_subset.groupby("wavelength").get_group("nm_190").pipe(
    lambda df: so.Plot(df, x="i").add(so.Line(), y="bcorr").add(so.Dot(), y="peaks")
)

In [ ]:
m_d111_subset.groupby('wavelength')['peaks'].agg(lambda x: x.dropna().shape[0]).plot()

So yeah, there is a disconnect between the expected components and the number of peaks. Ah well, pushing on.

# SIMPLISMA

SIMPLe-to-use Interactive Self-modeling Mixture Analysis.

Selection of pure variables from $D$.

First published by @windig_1991.

In [ ]:
m_d111_subset

In [ ]:
#Main Algorithm
def simplisma(d, nr, error):

	def wmat(c,imp,irank,jvar):
		dm=np.zeros((irank+1, irank+1))
		dm[0,0]=c[jvar,jvar]
		
		for k in range(irank):
			kvar=int(imp[k])
			
			dm[0,k+1]=c[jvar,kvar]
			dm[k+1,0]=c[kvar,jvar]
			
			for kk in range(irank):
				kkvar=int(imp[kk])
				dm[k+1,kk+1]=c[kvar,kkvar]
		
		return dm

	nrow,ncol=d.shape
	
	dl = np.zeros((nrow, ncol))
	imp = np.zeros(nr)
	mp = np.zeros(nr)
	
	w = np.zeros((nr, ncol))
	p = np.zeros((nr, ncol))
	s = np.zeros((nr, ncol))
	
	error=error/100
	mean=np.mean(d, axis=0)
	error=np.max(mean)*error
	
	s[0,:]=np.std(d, axis=0)
	w[0,:]=(s[0,:]**2)+(mean**2)
	p[0,:]=s[0,:]/(mean+error)

	imp[0] = int(np.argmax(p[0,:]))
	mp[0] = p[0,:][int(imp[0])]
	
	l=np.sqrt((s[0,:]**2)+((mean+error)**2))

	for j in range(ncol):
		dl[:,j]=d[:,j]/l[j]
		
	c=np.dot(dl.T,dl)/nrow
	
	w[0,:]=w[0,:]/(l**2)
	p[0,:]=w[0,:]*p[0,:]
	s[0,:]=w[0,:]*s[0,:]
	
	print('purest variable 1: ', int(imp[0]+1), mp[0])

	for i in range(nr-1):
		for j in range(ncol):
			dm=wmat(c,imp,i+1,j)
			w[i+1,j]=np.linalg.det(dm)
			p[i+1,j]=w[i+1,j]*p[0,j]
			s[i+1,j]=w[i+1,j]*s[0,j]
			
		imp[i+1] = int(np.argmax(p[i+1,:]))
		mp[i+1] = p[i+1,int(imp[i+1])]
		
		print('purest variable '+str(i+2)+': ', int(imp[i+1]+1), mp[i+1])
		
	sp=np.zeros((nrow, nr))
			
	for i in range(nr):
		sp[0:nrow,i]=d[0:nrow,int(imp[i])]
		
	plt.subplot(3, 1, 2)
	plt.plot(sp)
	plt.title('Estimate Components')
	
	concs = np.dot(np.linalg.pinv(sp), d)
	
	plt.subplot(3, 1, 3)
	for i in range(nr):
		plt.plot(concs[i])
	plt.title('Concentrations')
	plt.show()
	
	return sp, concs

m_d111_subset_aug = m_d111_subset.pivot_table(columns='wavelength',index='i',values='scale_center')
#Run Simplisma
sp, concs = simplisma(m_d111_subset_aug.values, 5		, 5)


In [ ]:
sp.shape

In [ ]:
plt.plot(sp);

In [ ]:
plt.plot(m_d111_subset_aug);

In [ ]:
sp.shape

In [ ]:
m_d111_subset_aug.shape

# MCR

In [ ]:
import sys
import pymcr
import logging
from pymcr.mcr import McrAR
from pymcr.regressors import OLS, NNLS
from pymcr.constraints import ConstraintNonneg, ConstraintNorm

logger = logging.getLogger('pymcr')
logger.setLevel(logging.DEBUG)

# StdOut is a "stream"; thus, StreamHandler
stdout_handler = logging.StreamHandler(stream=sys.stdout)

mcrar=McrAR(max_iter=100,
            st_regr='OLS',
            c_regr='OLS',
            c_constraints=[ConstraintNonneg(), ConstraintNorm()],
            tol_increase=1e4,
            tol_n_above_min=10
            )
mcrar.fit(D=m_d111_subset_aug,
          ST=sp.T,
          verbose=True)

In [ ]:
copt = mcrar.C_opt_

In [ ]:
copt.shape

In [ ]:
plt.plot(m_d111_subset_aug);

In [ ]:
copt = pd.DataFrame(copt)
copt.head()

In [ ]:
m_copt = copt.melt(var_name='column', value_name='conc')
m_copt.head()

In [ ]:
plt.plot(mcrar.C_opt_.dot(mcrar.ST_opt_).T);

In [ ]:
out = pd.DataFrame(mcrar.C_opt_.dot(mcrar.ST_opt_), columns = m_d111_subset_aug.columns, index=m_d111_subset_aug.index)
out.head()

In [ ]:
m_d111_subset_aug.reset_index().melt(var_name='wavelength', id_vars='i',value_name='abs').groupby('wavelength').get_group('256')['abs'].plot()

In [ ]:
melt_out = out.reset_index().melt(var_name='wavelength', id_vars='i', value_name='abs')
melt_out.groupby('wavelength').get_group('256')['abs'].plot()

In [ ]:
melt_out.head()

In [ ]:
melt_out = melt_out.fillna(0)
melt_out.head()

In [ ]:
cc = plt.tricontourf(melt_out.wavelength, melt_out.i, melt_out['abs'], level=10)
artists, labels= cc.legend_elements()
plt.legend(artists, labels,bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
opath = "/Users/jonathan/mres_thesis/wine_analysis_hplc_uv/src/wine_analysis_hplc_uv/notebooks/pca_sample.parquet"
m_d111_subset_aug.to_parquet(opath)